In [1]:
import pandas as pd

In [57]:
input_sentiment = pd.read_csv("sentiment_data.csv", header=0, delimiter=",")
input_sentiment['s_indicator']=0
print (input_sentiment.shape)

(112197, 30)


In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
# Provisioning for part-of-speech tagging
def penn_to_wn(tag):
     if tag.startswith('J'):
        return wn.ADJ
     elif tag.startswith('N'):
        return wn.NOUN
     elif tag.startswith('R'):
        return wn.ADV
     elif tag.startswith('V'):
        return wn.VERB
     return None

In [7]:
def swn_polarity(text):
    sentiment = 0
    tokens_count = 0
    
    for raw_sentence in text:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            #Assigning sentiment score
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
            
            if not tokens_count:
                return 0
            if sentiment >= 0:
                return 1
            

            return 0

In [8]:
#taking users review data for analysis
review_data = input_sentiment["reviewText"]

In [14]:
lsentiment=[]

for i in review_data:
    lsentiment.append(swn_polarity(i))
    resultsentiment=zip(review_data,lsentiment)
    input_sentiment['s_indicator']=pd.Series(lsentiment)
done

In [15]:
#Performing Regression (Logistic) on the data

In [16]:
import statsmodels.api as sm2

done


In [17]:
train = input_sentiment[:90000] 
test = input_sentiment[90000:]

In [32]:
#Data cleansing
train = train.fillna(0)
test=test.fillna(0)

In [47]:

logit = sm2.Logit(train['s_indicator'].dropna(),train['overall'].dropna())
result = logit.fit() 
print (result.summary())

Optimization terminated successfully.
         Current function value: 0.306467
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             lsentiment   No. Observations:                90000
Model:                          Logit   Df Residuals:                    89999
Method:                           MLE   Df Model:                            0
Date:                Mon, 27 Feb 2017   Pseudo R-squ.:                -0.05248
Time:                        15:51:59   Log-Likelihood:                -27582.
converged:                       True   LL-Null:                       -26207.
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
overall        0.5217      0.003    191.052      0.000         0.516     0.527


In [54]:
#prediction of sentiment based on confidence
predictSentiment = result.predict(test['overall'])

predictSentiment = (predictSentiment > 0.50).astype(int)
print(predictSentiment)

[1 1 1 ..., 1 1 1]


In [55]:
#Generating a confusion mmatrix
from sklearn.metrics import confusion_matrix
print (confusion_matrix(test['lsentiment'],predictSentiment))

Confusion Matrix
[[    0  1860]
 [    0 20337]]


In [56]:
#Deriving accuracy scores
from sklearn.metrics import accuracy_score
print(accuracy_score(test['s_indicator'],predictSentiment))

0.916204892553
